#### Анализ ботов

Предстоит сделать следующее:

- Оценить количество ботов, которые заходят на наш сайт, и их долю от всего трафика.
- Определить, с каких источников и в какое время суток чаще всего заходят роботы.

Примечание. Роботами могут быть пауки поисковых систем, которые индексируют контент блога, или же различные парсеры и агрегаторы контента. «Яндекс.Метрика» позволяет находить этих роботов и показывает их в отдельном параметрe ym:s:isRobot, а также считает их долю в специальной метрике ym:s:robotPercentage.

Идентификатор счетчика Яндекс.Метрики сайта: 30177909

In [1]:
import json 
import requests
from pprint import pprint
import pandas as pd

In [2]:
token = 'AQAAAAAPRPI2AAdDCsshW8VYzkqSnrKKpj8sJlE'
headers = {'Authorization': 'OAuth ' + token}

Возьмем 2 метрики - количество визитов и процент роботов. Сгруппируем их по параментрам - часы и трафик.
Данные по сентябрю 2020 года.

In [3]:
params = {'metrics': 'ym:s:visits,ym:s:robotPercentage',
          'dimensions': 'ym:s:hour,ym:s:lastsignTrafficSource',  
          'date1': '2020-09-01',  
          'date2': '2020-09-30',     
          'ids': 30177909,
          'accuracy':'full',
          'limit':100000}

In [4]:
'ym:s:goal137595631reaches'

'ym:s:goal137595631reaches'

In [5]:
response = requests.get('https://api-metrika.yandex.net/stat/v1/data', params=params, headers=headers)
print (response.status_code)

200


In [6]:
bots = response.json()
bots

{'query': {'ids': [30177909],
  'dimensions': ['ym:s:hour', 'ym:s:lastsignTrafficSource'],
  'metrics': ['ym:s:visits', 'ym:s:robotPercentage'],
  'sort': ['-ym:s:visits'],
  'date1': '2020-09-01',
  'date2': '2020-09-30',
  'limit': 100000,
  'offset': 1,
  'group': 'Week',
  'auto_group_size': '1',
  'attr_name': '',
  'quantile': '50',
  'offline_window': '21',
  'attribution': 'LastSign',
  'currency': 'RUB',
  'adfox_event_id': '0'},
 'data': [{'dimensions': [{'id': '16', 'name': '16:00'},
    {'icon_id': '2',
     'icon_type': 'traffic-source',
     'name': 'Search engine traffic',
     'id': 'organic'}],
   'metrics': [251.0, 24.70119522]},
  {'dimensions': [{'id': '15', 'name': '15:00'},
    {'icon_id': '2',
     'icon_type': 'traffic-source',
     'name': 'Search engine traffic',
     'id': 'organic'}],
   'metrics': [240.0, 26.25]},
  {'dimensions': [{'id': '11', 'name': '11:00'},
    {'icon_id': '2',
     'icon_type': 'traffic-source',
     'name': 'Search engine traffic',
 

Данные по эти датам имеются, теперь посмотрим ближе на статистику по ключу data.

In [7]:
pprint(bots['data'][0:5])

[{'dimensions': [{'id': '16', 'name': '16:00'},
                 {'icon_id': '2',
                  'icon_type': 'traffic-source',
                  'id': 'organic',
                  'name': 'Search engine traffic'}],
  'metrics': [251.0, 24.70119522]},
 {'dimensions': [{'id': '15', 'name': '15:00'},
                 {'icon_id': '2',
                  'icon_type': 'traffic-source',
                  'id': 'organic',
                  'name': 'Search engine traffic'}],
  'metrics': [240.0, 26.25]},
 {'dimensions': [{'id': '11', 'name': '11:00'},
                 {'icon_id': '2',
                  'icon_type': 'traffic-source',
                  'id': 'organic',
                  'name': 'Search engine traffic'}],
  'metrics': [228.0, 25.87719298]},
 {'dimensions': [{'id': '13', 'name': '13:00'},
                 {'icon_id': '2',
                  'icon_type': 'traffic-source',
                  'id': 'organic',
                  'name': 'Search engine traffic'}],
  'metrics': [227.0, 2

Приведем словарь к Датафрейму.

In [8]:
bots2 = pd.DataFrame(bots['data'])
display(bots2.head(10))

,dimensions,metrics
0,"[{'id': '16', 'name': '16:00'}, {'icon_id': '2...","[251.0, 24.70119522]"
1,"[{'id': '15', 'name': '15:00'}, {'icon_id': '2...","[240.0, 26.25]"
2,"[{'id': '11', 'name': '11:00'}, {'icon_id': '2...","[228.0, 25.87719298]"
3,"[{'id': '13', 'name': '13:00'}, {'icon_id': '2...","[227.0, 28.1938326]"
4,"[{'id': '12', 'name': '12:00'}, {'icon_id': '2...","[225.0, 24.88888889]"
5,"[{'id': '14', 'name': '14:00'}, {'icon_id': '2...","[225.0, 25.77777778]"
6,"[{'id': '17', 'name': '17:00'}, {'icon_id': '2...","[209.0, 26.31578947]"
7,"[{'id': '10', 'name': '10:00'}, {'icon_id': '2...","[186.0, 19.35483871]"
8,"[{'id': '21', 'name': '21:00'}, {'icon_id': '2...","[166.0, 17.46987952]"
9,"[{'id': '19', 'name': '19:00'}, {'icon_id': '2...","[161.0, 23.60248447]"


Информацию в таком фиде сложно анализировать, поэтому избавимся от вложенных словарей.

In [9]:
def NoDicts(bots):
    list_of_dicts = []
    dimensions_list = bots['query']['dimensions']
    metrics_list = bots['query']['metrics']
    for item in bots['data']:
        d = {}
        for i,dimension in enumerate(item['dimensions']):
            d[dimensions_list[i]] = dimension['name']
        for i,metric in enumerate(item['metrics']):
            d[metrics_list[i]] = metric
        list_of_dicts.append(d)
    return list_of_dicts

In [10]:
bots_list = NoDicts(bots)
pprint(bots_list[0:5])

[{'ym:s:hour': '16:00',
  'ym:s:lastsignTrafficSource': 'Search engine traffic',
  'ym:s:robotPercentage': 24.70119522,
  'ym:s:visits': 251.0},
 {'ym:s:hour': '15:00',
  'ym:s:lastsignTrafficSource': 'Search engine traffic',
  'ym:s:robotPercentage': 26.25,
  'ym:s:visits': 240.0},
 {'ym:s:hour': '11:00',
  'ym:s:lastsignTrafficSource': 'Search engine traffic',
  'ym:s:robotPercentage': 25.87719298,
  'ym:s:visits': 228.0},
 {'ym:s:hour': '13:00',
  'ym:s:lastsignTrafficSource': 'Search engine traffic',
  'ym:s:robotPercentage': 28.1938326,
  'ym:s:visits': 227.0},
 {'ym:s:hour': '12:00',
  'ym:s:lastsignTrafficSource': 'Search engine traffic',
  'ym:s:robotPercentage': 24.88888889,
  'ym:s:visits': 225.0}]


Создаем теперь датафрейм с вытащенными данными.

In [11]:
bots_df = pd.DataFrame(bots_list)
bots_df.columns=['Day time','Source','Visits','Bots percentage']
display(bots_df.head(10))

,Day time,Source,Visits,Bots percentage
0,16:00,Search engine traffic,251.0,24.701195
1,15:00,Search engine traffic,240.0,26.250000
2,11:00,Search engine traffic,228.0,25.877193
3,13:00,Search engine traffic,227.0,28.193833
4,12:00,Search engine traffic,225.0,24.888889
5,14:00,Search engine traffic,225.0,25.777778
6,17:00,Search engine traffic,209.0,26.315789
7,10:00,Search engine traffic,186.0,19.354839
8,21:00,Search engine traffic,166.0,17.469880
9,19:00,Search engine traffic,161.0,23.602484


Теперь можно сгруппировать датафрейм по времени суток и отсортировать по проценту ботов, чтобы понять, в какой части дня их больше всего.

In [12]:
bots_by_daytime = bots_df.groupby('Day time').agg({'Visits': 'sum' , 'Bots percentage' : 'mean'})
bots_by_daytime.sort_values('Bots percentage', ascending = False)

,Visits,Bots percentage
Day time,,
03:00,38.0,44.312169
16:00,310.0,43.035477
22:00,190.0,34.579012
13:00,273.0,27.918994
09:00,203.0,26.106668
12:00,303.0,25.165701
10:00,240.0,25.071268
23:00,157.0,24.107407
06:00,53.0,21.524064


#### Вывод: В среднем больше всего доля ботов приходится на 3 часа ночи и 4 часа дня - 43-44% от всех визитов  в это время.

Теперь посмотрим на источники трафика и откуда идет больше всего ботов там:

In [13]:
bots_by_source = bots_df.groupby('Source').agg({'Visits': 'sum' , 'Bots percentage' : 'mean'})
bots_by_source.sort_values('Bots percentage', ascending = False)

,Visits,Bots percentage
Source,,
Link traffic,126.0,27.249606
Internal traffic,10.0,25.000000
Direct traffic,757.0,23.835393
Search engine traffic,3334.0,18.963520
Social network traffic,80.0,11.084055
Cached page traffic,6.0,10.000000
Recommendation system traffic,2.0,0.000000


Больше 20% роботов приходят из ссылок на других сайтах, внутренних переходов и прямых заходов на сайт.

Здесь смущают, наверное, внутренние переходы Internal traffic, тк их всего 10 и четверть из них это боты. В определении Яндекса об этом трафике пишется следующее: "Если активность пользователя на сайте прервалась и тайм-аут визита закончился, но окно браузера осталось открытым, то при возобновлении активности будет создан новый визит с источником «внутренний переход»." Так что возможно это бот, который пришел из другого трафика, но "завис" на сайте.